In [ ]:
!pip install torch torchvision torchaudio numpy matplotlib h5py pandas scikit-learn scipy tqdm
!pip install git+https://github.com/AurelienDecelle/TorchRBM.git

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

from rbm.binary.functions import init_chains
from rbm.binary.sampling import sample_state,sample_hiddens
from rbm.binary.train import train
from rbm.custom_fn import compute_U
from rbm.dataset import DatasetRBM
from rbm.io import load_params
from rbm.plot import plot_PCA
from rbm.utils import get_eigenvalues_hystory
from rbm.utils import get_checkpoints as get_checkpoints_trained
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Found device: {device}")

In [ ]:
# Some utility functions
def ImConcat(X,ncol=10,nrow=5,sx=28,sy=28):
        tile_X = []
        for c in range(nrow):
            L = torch.cat((tuple(X[i,:].reshape(sx,sy) for i in np.arange(c*ncol,(c+1)*ncol))))
            tile_X.append(L)
        return torch.cat(tile_X,1)
def plot_image(sample, shape=(28, 28), grid_size=(10, 10), show_grid=False):
    num_samples = grid_size[0] * grid_size[1]
    id_sample = np.random.randint(0, sample.shape[0], num_samples)
    display = np.zeros((shape[0] * grid_size[0], shape[1] * grid_size[1]))
    for i, id_s in enumerate(id_sample):
        idx, idy = i % grid_size[0], i // grid_size[1]
        display[
            (idx * shape[0]) : ((idx + 1) * shape[0]),
            (idy * shape[1]) : (idy + 1) * shape[1],
        ] = sample[id_s].reshape(shape[0], shape[1])
    fig, ax = plt.subplots(1, 1)
    ax.imshow(display, cmap="gray")
    if show_grid:
        # Minor ticks
        ax.set_xticks(np.arange(-0.5, grid_size[0] * shape[0], shape[0]), minor=True)
        ax.set_yticks(np.arange(-0.5, grid_size[1] * shape[1], shape[1]), minor=True)

        # Gridlines based on minor ticks
        ax.grid(which="minor", color="gray", linestyle="-", linewidth=2)
        ax.imshow(display, cmap="gray")
    else:
        ax.axis("off")

def get_checkpoints(n_save: int, num_updates: int, spacing: str = "exp"):
    if spacing == 'exp':
        checkpoints = []
        xi = num_updates
        for _ in range(n_save):
            checkpoints.append(xi)
            xi = xi / num_updates**(1 / n_save)
        checkpoints = np.unique(np.array(checkpoints, dtype=np.int32))
    elif spacing == 'linear':
        checkpoints = np.linspace(1, num_updates, n_save).astype(np.int32)
    checkpoints = np.unique(np.append(checkpoints, num_updates))
    return checkpoints


# Create the dataset

Colab provides us with several datasets, among which we can start with [MNIST](http://yann.lecun.com/exdb/mnist/) which contains 28$\times$28 pixel images of handwritten digits in white and black using variables that range from 0 to 255. Original data is in grey scale, but we will be working with binary black/white (or 0, 1 for us) variables. For this reason, we rewrite the dataset so that all pixels with $I/255>0.3$ is 1.

In [ ]:
my_data = np.genfromtxt('sample_data/mnist_train_small.csv', delimiter=',')
labels, dataset = my_data[:,0], my_data[:,1:]
print(labels)
# Convert to binary dataset
dataset /= 255.
data = np.array(dataset > 0.3, dtype=float)
# write to .dat
np.savetxt("MNIST.dat", data)
data=torch.tensor(data,device=device)

We show a subsample of all the images

In [ ]:
plot_image(dataset)

<font color='red'>

This is a standard data set for classification applications. For this reason, we also know the digit represented in each image, even if we will not use it for our training, as we will be working in a completely unsupervised setup.
1. Visualize the first entry of our thresholded dataset with `plt.imshow()`. Our data are vectors with 784 entries. In order to display them as an image, we need to transform them into a $28\times 28$ tensor. You can do this with `.reshape(28,28)`. You may need to transform the data to `.cpu()` before plotting it.
2. `print` the corresponding label

</font>


### dimensional reduction PCA

We can reduce the dimension of the dataset (784) to visualize it in lower dimensions. For this goal, we need to obtain the principal directions of the dataset which are nothing but the eigenvectors of the covariance matrix of the data. 

<font color='red'>

1. Our dataset $X$ is a $M\times d$ matrix, where $M$ is the number of samples and $d$ the dimension of our data. In this way, the entry $x_{mi}$ contains the $i$th pixel in the $m$-th example. Which are our $M$ and $d$? You can check the dimension of a tensor `X` using `X.shape`.

</font>

<font color='red'>

3. Compute the empirical covariance matrix of the data
$$\boldsymbol Q=\frac{1}{M-1} (\boldsymbol X-\overline{\boldsymbol X})^\top (\boldsymbol X-\overline{\boldsymbol X})\quad \text{or}\quad Q_{ij}=\frac{1}{M-1} \sum_{m=1}^M (x_{mi}-\overline{x_i})(x_{mj}-\overline{x_j})$$
You can use 

`torch.cov(data.T)`

or do it yourself using matrix multiplications. Which should be its dimension?

</font>

<font color='red'>

3. Diagonalize the Q matrix with 


`Σ,U= torch.linalg.eig(Q)`

    
where `Σ` is a vector containing all the eigenvalues, and `U` the matrix that contains the corresponding eigenvectors in its columns, and
$Q=U^{-1} \Sigma U$. Transform `U=U.double()` to avoid dealing with complex variables


</font>


<font color='red'>
    
4. Visualize the first 5 principal directions (the eigenvectors with the 5 largest eigenvalues) as images using `plt.imshow()`. The `i`-th column of a bidimensional tensor is extracted as `U[:,i]`

</font>

In [ ]:
fig, ax = plt.subplots(1,5)
for i in range(5):
    ax[i].imshow(#####.reshape(28,28).cpu(), cmap="gray")
plt.show()

<font color='red'>
    
5. Project each entry of the dataset along the first two principal components and show them in a scatter plot using the code below. 
</font>

In [ ]:
proj=(data@U).cpu()

fig, ax = plt.subplots()
scatter=plt.scatter(###,marker='.',c=labels,cmap='Spectral')
legend1 = ax.legend(*scatter.legend_elements(),loc="lower right", title="labels",ncols=2)
ax.add_artist(legend1)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()


## Training

We will perform a very short training process. First, we need to set the hyperparameters. We will take them into account:
- $N_\mathrm{h}=100$ hidden nodes
- $10^4$ parameter updates
- Use 2000 parallel chains to estimate the MCMC averages. Each time we change the parameters, we will perform $k=10$ steps.
- At the moment we use the persistent contrastive divergence (PCD) recipe. This means that every time we change the parameters, we restart the Markov chains at the last configurations reached in the previous update. We will try other schemes later. 

In [ ]:
# Hyperparameters

num_hiddens = 100 # no. of hidden nodes

num_updates = 10_000 # no. of parameter updates

learning_rate = 0.05    # lr 
batch_size = 2000       # minibatch for the stochastic gradient descent 
num_chains = batch_size # no. of chains for the MCMC 
gibbs_steps = 10        # no. of Gibbs updates 

training_scheme="PCD" # we can choose among "PCD", "Rdm", "CD" 

checkpoints = get_checkpoints(n_save=100, num_updates=num_updates) # we want to save several machines (100) along the training trajectory

device = "cuda"

dataset = DatasetRBM("MNIST.dat", device=device)

We run the training process and save the evolution of the parameters in the file `RBMPCD.h5`

In [ ]:
train(
    filename="RBM"+training_scheme+".h5",
    dataset=dataset,
    num_updates=num_updates,
    num_hiddens=num_hiddens,
    training_mode=training_scheme,
    lr=learning_rate,
    batch_size=batch_size,
    num_chains=num_chains,
    gibbs_steps=gibbs_steps,
    checkpoints=checkpoints,
    device=device
)

File `RBM.h5` contains all the information about the training process. We can extract the samples in the permanent chain from `f["parallel_chains"][()]`

In [ ]:
fname_data = "MNIST.dat"
fname_model = "RBMPCD.h5"
with h5py.File(fname_model, "r") as f:
    for key in f["hyperparameters"].keys():
        print(key, f["hyperparameters"][key][()])
        
    num_hiddens = f["hyperparameters"]["num_hiddens"][()]
    chains = torch.tensor(f["parallel_chains"][()], device=device)


<font color='red'> 
    
Let's give a look to the permanent chain. 
     
2. Which is the dimension of the permanent chain `chains`?
3. Visualize the different parallel chains as images using the function `plot_image` used above for the dataset.
</font>

<font color='red'> 

As discussed in the class, we can use different recipes to initialize the MCMC simulations we use to compute the gradient. In particular, we are going to compare the performance for:
- (PCD) Persistent Contrastive Divergence. We maintain a "permanent chain", i.e., we always iterate on the same chain, or in other words, we initialize the Markov chain process at each update on the last configurations reached at the previous update.
- (Rdm): We always initialize the chains with random conditions.
- (CD): We always initialize the chains with the samples in the minibatch

1. We had already run a short training using PCD. Rerun the training process using the Rdm and CD recipes. The different models will be stored in different files `RBMPCD.h5`, `RBMPCD.h5`, `RBMCD.h5`.
2. Compare the images of the file chain obtained with each method, which corresponds to the last configurations generated in the last training update. 
3. Project the chain on the first two principal directions of the dataset

`data_proj = data @ U`

`chains_proj = chains @ U`

`plot_PCA(data_proj.cpu().numpy(), chains_proj.cpu().numpy(), labels=["dataset", f"Last chains"])`

4. Do you see any particular difference?
</font>

In [ ]:
for rec in ['PCD','Rdm','CD']:
    
    fname_model = "RBM"+rec+".h5"
    with h5py.File(fname_model, "r") as f:
        chains = torch.tensor(f["parallel_chains"][()], device=device)
        
        chains_proj = ###

        ######

        plot_PCA(data_proj.cpu().numpy(), chains_proj.cpu().numpy(), labels=["dataset", f"Last chains"])

## Generating Samples

Once we have trained the models, we can generate new samples through a MCMC process.  We select the parameters of the most trained model, initialize a set of `n_gen` parallel samples, and run a sampling process of `mcmc_steps`. 

<font color='red'> 

1. Using the PCD training, try to check how much time you need to thermalize the chains and start to generate samples with the code below. We can visualize the samples both by looking at the images, or by looking at their PCA.

2. We can either visualize the visible units by selecting the key `v` or the magnetizations `mv`. 
</font>

In [ ]:
rec='CD'
fname_model = "RBM"+rec+".h5"

n_gen = 5000
mcmc_steps = 10
checkpoint = get_checkpoints_trained(fname_model)[-1] # we select the last update saved in the folder
params = load_params(fname=fname_model, device=device, checkpoint=checkpoint) # extract the parameters of the model

chains_init = init_chains(num_chains=n_gen, num_visibles=num_visibles, num_hiddens=num_hiddens, device=device) # initialize ramdomly the chains


gen = sample_state(chains=chains_init, params=params, gibbs_steps=mcmc_steps)["mv"] # sample the model for mcmc_steps and extract the configurations

gen_proj = gen @ U.float() 

plot_image(gen.cpu().numpy())
plot_PCA(data_proj.cpu().numpy(), gen_proj.cpu().numpy(), labels=["dataset", f"{mcmc_steps} generation steps"])

We are going to visualize the evolution of 15 parallel independent chains during the sampling process using the code below in which we show the images we generate as a function of time for times that grow exponentially.

In [ ]:
n_gen=num_chains
checkpoint = get_checkpoints_trained(fname_model)[-1]
params = load_params(fname=fname_model, device=device, checkpoint=checkpoint)
chains_init = init_chains(num_chains=n_gen, num_visibles=num_visibles, num_hiddens=num_hiddens, device=device)


all_vis_datatest=[]
vis =chains_init
all_vis_datatest.append(vis["v"])


x3=np.unique(np.logspace(0,5,base=10,num=16,dtype=int))

t_tot=0

for t in x3:
    #print(t)
    Δt = t-t_tot
    state=sample_state(chains=vis, params=params, gibbs_steps=int(Δt))
    vis=state
    C_gen=torch.cov(state["v"].T)
    
    m_vis=state["mv"]
    t_tot += Δt
    all_vis_datatest.append(m_vis)
    


times=[0]
times.extend(x3)

all_vis_1=all_vis_datatest
f,ax = plt.subplots(len(all_vis_1),dpi=100)
for i in range(len(all_vis_1)):
    Im = ImConcat(all_vis_1[i].cpu(),ncol=1,nrow=15)
    ax[i].imshow(Im,cmap='gray')
    ax[i].set_xticks([], [])
    ax[i].set_yticks([], [])
    ax[i].text(5, 5, str(int(times[i])), bbox={'facecolor': 'white', 'pad':1},fontsize=5) #: 5,fontsize : 5})
plt.show()

<font color='red'>

3. Repeat the same process using the other two training recipes. What difference do you observe? How much time do you need to generate numbers with each recipe?

4. What happens now if you change the initialization of the chains. For instance, we can initialize the Markov chains on samples of the dataset using
   `chains_init = {"v" : dataset.data[:n_gen]}`
   
   `chains_init = sample_hiddens(chains_init, params)`

5. Let us try to quantify the quality of the samples in sampling time. For that, we can compute the Pearson correlation between the dataset and the generated set covariance matrix. You can compute it for each time using

   `torch.corrcoef(torch.vstack([C_data.reshape(-1), C_gen.reshape(-1)]))[0, 1].cpu()`

    Plot the evolution of the Pearson in time for the three training processes.

6. For the `Rdm` recipe prove a change the number of Gibbs steps used to train the machine. Where does the best quality is generated?
</font>

In [ ]:
for rec in ['PCD','CD','Rdm']:
    
    fname_model = "RBM"+rec+".h5"
    n_gen=num_chains
    checkpoint = get_checkpoints_trained(fname_model)[-1]
    params = load_params(fname=fname_model, device=device, checkpoint=checkpoint)
    chains_init = init_chains(num_chains=n_gen, num_visibles=num_visibles, num_hiddens=num_hiddens, device=device)
    
    
    all_vis_datatest=[]
    vis =chains_init
    all_vis_datatest.append(vis["v"])
    
    
    x3=np.unique(np.logspace(0,5,base=10,num=14,dtype=int))
    
    t_tot=0
    ##
    for t in x3:
        #print(t)
        Δt = t-t_tot
        state=sample_state(chains=vis, params=params, gibbs_steps=int(Δt))
        vis=state
        C_gen=torch.cov(state["v"].T)
        
        
        #####
        
        m_vis=state["mv"]
        t_tot += Δt
        all_vis_datatest.append(m_vis)
        
    pearson_coef=np.array(pearson_coef)
    plt.plot(x3,pearson_coef,label=rec)

plt.ahline(y=10,label=r'$k=10$ training')
plt.xscale('log')
plt.legend()
plt.show()

## Training dynamics and phase transitions

The training of the RBMs can be very well tracked using the singular value decomposition of the model weight matrix. We show below the evolution of the singular values (the eigenvalues for a rectangular matrix) in training time. 

In [ ]:
rec="PCD"
fname_model = "RBM"+rec+".h5"

updates, eigenvalues = get_eigenvalues_hystory(fname_model)

fig, ax = plt.subplots(dpi=100, nrows=1, ncols=1, figsize=(6, 3))
ax.set_title('weight matrix eigenvalues', size=15)
ax.set_xlabel('gradient updates')
ax.set_ylabel(r'$w_\alpha$')
ax.set_xscale('log')
ax.set_yscale('log')
ax.grid(alpha=0.5, ls='dashed')
ax.plot(updates, eigenvalues);

We will see that at the first stages of the training, the singular vectors of the W matrix align with the principal components of the dataset. 

<font color='red'> 

1. Show the first 5 principal components of the dataset as done above and compare them with the first 5 singular vectors of W at t=99. What happens at later times?

</font>



In [ ]:
f = h5py.File(fname_model, 'r')
it=99
key='update_'+str(it)
weight_matrix = torch.tensor(f[key]["weight_matrix"][()],device=device)

UU,w,VV=torch.svd(weight_matrix)

fig, ax = plt.subplots(1,5)
fig.suptitle('PCA components',y=.7)
for i in range(5):
    ax[i].imshow(###.cpu().reshape(28,28), cmap="gray")
fig.tight_layout()

plt.show()

fig, ax = plt.subplots(1,5)
fig.suptitle('singular vectors',y=.7)
for i in range(5):
    
    ax[i].imshow(###.cpu().reshape(28,28), cmap="gray")
fig.tight_layout()
plt.show()


f.close()


We will see in class that the training process can be seen as a sequence of phase transitions. For this reason, for different number of training updates, we will compute the susceptibility associated to the magnetization of the data projected on the first 5 singular vectors of W ($\boldsymbol{u}_\alpha$ with $\alpha=1,\ldots,5$) as
   $$\chi_\alpha(t)=N \left(\overline{m_\alpha^2}-\overline{m}^2\right)$$
with
$$m_\alpha= \frac{1}{\sqrt{N}}\sum_i u_{i}^\alpha v_i$$
and where the averages are computed using the parallel chains.


In [ ]:

mcmc_steps=100
n_gen = 5000

chains_init = init_chains(num_chains=n_gen, num_visibles=num_visibles, num_hiddens=num_hiddens, device=device)
chains=chains_init

pearson_coef=[]
proj=[]
eigenvalues=[]
for it in updates:
    
    params = load_params(fname=fname_model, device=device, checkpoint=it)
    weight_matrix=params["weight_matrix"]
    UU,w,VV=torch.svd(weight_matrix)
    eigenvalues.append(np.array(w.cpu()))
    
    chains = sample_state(chains=chains, params=params, gibbs_steps=mcmc_steps)
    
    m=chains["v"]@UU.float()/np.sqrt(num_visibles)
    
    chi=torch.mean((m-m.mean(0))**2,0)
    proj.append(np.array(chi.cpu())*num_visibles)

fig,ax=plt.subplots(2,1,sharex=True)
eigenvalues=np.array(eigenvalues)
ax[0].plot(updates,eigenvalues[:,:5])
ax[0].set_xlabel('updates')
ax[0].set_ylabel(r'$w_\alpha$')
proj=np.array(proj)

ax[1].plot(updates,proj[:,:5])
plt.legend()
plt.xscale('log')
ax[1].set_xlabel('updates')
ax[1].set_ylabel(r'$\chi_\alpha$')
plt.show()



<font color='red'> 

2. Plot the susceptibility along each mode against the singular value along that mode.
</font>

In [ ]:
###